In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np


wdir = "/home/jose/Dropbox/biblia/tb/"
file = "TEIBible" # "*.xml"
outdir = "/home/jose/Dropbox/biblia/tb/resulting data/"

In [44]:
books_df = pd.ExcelFile(wdir+"/documentation/books.xlsx").parse('Sheet1').fillna("")

In [45]:
books_df.head()

,bible,id,libro,testamento,encoded,english,codebook,Chaptersc,genre,latitude,longitude
0,bible,1,Génesis,antiguo,1,Genesis,GEN,50.0,historical,32.6157,36.7931
1,bible,2,Éxodo,antiguo,1,Exodus,EXO,40.0,historical,30.47,33.8433
2,bible,3,Levítico,antiguo,1,Leviticus,LEV,27.0,law,30.5218,34.1126
3,bible,4,Números,antiguo,0,Numbers,NUM,36.0,law,,
4,bible,5,Deuteronomio,antiguo,0,Deuteronomy,DEU,34.0,law,,


In [46]:
metadata = pd.ExcelFile(wdir+"entities.xls").parse('Sheet1').fillna("")
metadata.index = metadata["id"]

In [47]:
metadata.head()

,id,NormalizedName-sp,aclaration,variants,comments,book,cert,Gender,NormalizedName-ge,NormalizedName-po,...,PHM,HEB,JAM,1PE,2PE,1JO,2JO,3JO,JUD,REV
id,,,,,,,,,,,,,,,,,,,,,
#per0,#per0,persona no definida,,,,NT,,none,nicht identifizierte Person,pessoa não identificada,...,0,0,0,0,0,0,0,0,0,0
#per1,#per1,Jesús,,,,NT,,male,Jesus,Jesus,...,13,75,5,34,24,48,7,0,8,102
#per2,#per2,Abraham,,Abram,,AT,,male,Abraham,Abraão,...,0,13,4,2,0,0,0,0,0,0
#per3,#per3,Isaac,,,,AT,,male,Isaak,Isaque,...,0,7,2,0,0,0,0,0,0,0
#per4,#per4,Jacob,,Israel,,AT,,male,Jakob,Jacó,...,0,5,0,0,0,1,0,0,0,0


In [100]:
books_people_df = pd.DataFrame(columns=books_df.loc[books_df["encoded"]==1]["codebook"].tolist(), index=metadata.loc[metadata["type"]!="place"].index.tolist()).fillna("")

In [101]:
books_people_df.head()

,GEN,EXO,LEV,JOS,JDG,RUT,1SA,2SA,1KI,EZR,...,PHM,HEB,JAM,1PE,2PE,1JO,2JO,3JO,JUD,REV
#per0,,,,,,,,,,,...,,,,,,,,,,
#per1,,,,,,,,,,,...,,,,,,,,,,
#per2,,,,,,,,,,,...,,,,,,,,,,
#per3,,,,,,,,,,,...,,,,,,,,,,
#per4,,,,,,,,,,,...,,,,,,,,,,


In [121]:
book = "MAT"
matrix_units_entities = pd.read_csv("/home/jose/Dropbox/biblia/tb/resulting data/TEIBible_"+book+"_q-rs@key-@toWhom-@who__matrix.csv", sep="\t", index_col=0)

In [122]:
matrix_units_entities.head()

,b.MAT.1.1,b.MAT.1.2,b.MAT.1.3,b.MAT.1.4,b.MAT.1.5,b.MAT.1.6,b.MAT.1.7,b.MAT.1.8,b.MAT.1.9,b.MAT.1.10,...,b.MAT.28.11,b.MAT.28.12,b.MAT.28.13,b.MAT.28.14,b.MAT.28.15,b.MAT.28.16,b.MAT.28.17,b.MAT.28.18,b.MAT.28.19,b.MAT.28.20
#per1359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#per208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#pla260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#pla364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
places = sorted([entity for entity in matrix_units_entities.index.tolist() if "pla" in entity])

In [124]:
other_entities = sorted([entity for entity in matrix_units_entities.index.tolist() if "pla" not in entity])

In [125]:
matrix_entities_places = pd.DataFrame([], index=other_entities, columns=places).fillna(0)

In [126]:
matrix_entities_places.head()

,#pla0,#pla1,#pla100,#pla101,#pla102,#pla103,#pla104,#pla105,#pla106,#pla107,...,#pla90,#pla91,#pla92,#pla93,#pla94,#pla95,#pla96,#pla97,#pla98,#pla99
#ani0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
for textual_unit in matrix_units_entities.columns.tolist():
    entities_in_textual_unit_lt = sorted(matrix_units_entities[textual_unit][matrix_units_entities[textual_unit] > 0].index)
    
    if any("pla" in entity for entity in entities_in_textual_unit_lt):
        places_in_textual_unit_lt = [entity for entity in entities_in_textual_unit_lt if "pla" in entity]
        other_entities_in_textual_unit_lt = [entity for entity in entities_in_textual_unit_lt if "pla" not in entity]
        
        for entity in other_entities_in_textual_unit_lt:
            for place in places_in_textual_unit_lt:
                matrix_entities_places.loc[entity][place] += 1


In [128]:
matrix_entities_places.loc["#per1"].sort_values(ascending=False).head()

#pla19    38
#pla18    37
#pla15    11
#pla7     11
#pla3     10
Name: #per1, dtype: int64

In [129]:
matrix_entities_places

,#pla0,#pla1,#pla100,#pla101,#pla102,#pla103,#pla104,#pla105,#pla106,#pla107,...,#pla90,#pla91,#pla92,#pla93,#pla94,#pla95,#pla96,#pla97,#pla98,#pla99
#ani0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org10,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#org105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
matrix_entities_places.to_csv(outdir+"entities_places_"+book+".csv",sep="\t")

In [131]:
print(metadata.columns.tolist())

['id', 'NormalizedName-sp', 'aclaration', 'variants', 'comments', 'book', 'cert', 'Gender', 'NormalizedName-ge', 'NormalizedName-po', 'type', 'nature', 'definition', 'importance', 'order-edition', 'latitude', 'longitude', 'geo_cert', 'geo_controlled', 'sum_freq', 'median_freq', 'mean_freq', 'std_freq', 'GEN', 'EXO', 'LEV', 'JOS', 'JDG', 'RUT', '1SA', '2SA', '1KI', 'EZR', 'NEH', 'JOB', 'PSA', 'ECC', 'LAM', 'JER', 'EZE', 'DAN', 'HOS', 'JOE', 'AMO', 'OBA', 'JON', 'MIC', 'NAH', 'HAB', 'ZEP', 'HAG', 'ZEC', 'MAL', 'MAT', 'MAR', 'LUK', 'JOH', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'PHI', '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAM', '1PE', '2PE', '1JO', '2JO', '3JO', 'JUD', 'REV']


In [132]:
matrix_long = matrix_entities_places.copy()
for place in matrix_entities_places.columns.tolist():
    if len(metadata.loc[[place]].loc[metadata.loc[[place]]["geo_cert"].isin(["high","medium"])]) > 0:
        matrix_long.loc[ matrix_long[place] >= 1  , place] = str(metadata.loc[place]["latitude"]) +","+ str(metadata.loc[place]["longitude"]) 

In [153]:
import math
i =0
for entity in matrix_entities_places.index.tolist():
    longitude = np.array([float(item.split(",")[0]) for item in matrix_long.loc[entity].tolist() if type(item) is str ]).mean()
    latitude = np.array([float(item.split(",")[1]) for item in matrix_long.loc[entity].tolist() if type(item) is str ]).mean()
    if math.isnan(longitude):
        pass
    else:
        books_people_df.loc[entity,book] = str(longitude) + "," + str(latitude)


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:4: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: Mean of empty slice.


In [158]:
books_people_df.loc[books_people_df[book]!=""][book].shape

(43,)